# The Graph Data Access

In this notebook, we retrieve live Rai system data from [The Graph](https://thegraph.com/explorer/subgraph/reflexer-labs/rai-mainnet?query=Example%20query) and save it as a CSV for analysis in the [Graph Analysis](TheGraphDataAnalysis.ipynb) notebook.

## Full Reflexer Labs dashboad query
```
{
  internalCoinBalances(where: {accountHandler: "${GEB_STABILITY_FEE_TREASURY}"}) { 
    balance
  }
  accountingEngine(id: "current") { # can it use block number?
    activeDebtAuctions
    activeSurplusAuctions
  }
  collateralType(id: "ETH-A") {
    currentPrice {
      value
    }
    currentMedianizerUpdate {
      value
    }
    debtCeiling
    activeLiquidations
    liquidationsStarted
    totalCollateral
    totalAnnualizedStabilityFee
  }
  systemState(id: "current") { # can it use block number?
    coinUniswapPair {
      reserve0
      reserve1
    }
    currentCoinMedianizerUpdate{
      value
    }
    currentRedemptionRate {
      eightHourlyRate
      annualizedRate
      hourlyRate
      createdAt
    }
    currentRedemptionPrice {
      value
    }
    erc20CoinTotalSupply
    globalDebt
    globalDebtCeiling
    totalActiveSafeCount
    coinAddress
    wethAddress
    systemSurplus
    debtAvailableToSettle
  }
  dailyStats(first: 1000, orderDirection: desc) {
    globalDebt
    timestamp
    redemptionRate {
      twentyFourHourlyRate
    }
  }
  hourlyStats(first: 1000, orderBy: timestamp, orderDirection: desc) {
    globalDebt
    timestamp
    redemptionRate {
      hourlyRate
    }
    redemptionPrice{
      value
    }
    marketPriceUsd
  }
}
```

In [38]:
# import libraries
import pandas as pd
import json
import requests
import matplotlib.pyplot as plt

# Graph
url = 'https://api.thegraph.com/subgraphs/name/reflexer-labs/rai-mainnet'

## Hourly stats

In [39]:
query_header = '''
query {{
    hourlyStats(first: 1000, skip:{}) {{'''

query_tail = '''    
}
}'''

query_body = '''
 timestamp
    blockNumber
    redemptionRate {
      hourlyRate
    }
    redemptionPrice {
      id
      block
      value
      
    }
    marketPriceUsd # price of COIN in USD (uni pool price * ETH median price)
    marketPriceEth # Price of COIN in ETH (uni pool price)
    globalDebt
    erc20CoinTotalSupply
'''


n = 0
hourly = []
while True:
    print(f'request {n+1}')
    query = query_header.format(n*1000) + query_body + query_tail
    r = requests.post(url, json = {'query':query})
    s = json.loads(r.content)['data']['hourlyStats']
    print(f'results {len(s)}')
    hourly.extend(s)
    print(f'total {len(hourly)}')
    n += 1
    if len(s) < 1000:
        break
        
hourlyStats = pd.DataFrame(hourly)

request 1
results 851
total 851


In [40]:
hourlyStats

,blockNumber,erc20CoinTotalSupply,globalDebt,marketPriceEth,marketPriceUsd,redemptionPrice,redemptionRate,timestamp
0,11857308,1988.440432971667032725,1988.577499489843641169436691184972,0.002589462210048543650949087456484599,4.670500166535429740591356399938391,"{'block': '11849022', 'id': '0xe313271188492c4...",{'hourlyRate': '1'},1613338681
1,11860793,2788.440432971667032725,2788.617717345781163523077653358556,0.002481355746535171105569862943787003,4.377049344386119132419461918241039,"{'block': '11860755', 'id': '0x40066774f6fb7d3...",{'hourlyRate': '1'},1613384616
2,11861040,2788.440432971667032725,2788.657935201718685876567866781341,0.002481355746535171105569862943787003,4.406833024856393396658172078078713,"{'block': '11861008', 'id': '0x0d551e2a2d55fa8...",{'hourlyRate': '1'},1613387890
3,11861337,2788.440432971667032725,2788.657935201718685876567866781341,0.002481355746535171105569862943787003,4.419265710853707004204445293455575,"{'block': '11861237', 'id': '0x5c67dbb323db039...",{'hourlyRate': '1'},1613392101
4,11861474,3048.440432971667032725,3048.645954953510904626288422086589,0.001768934955701819449591545208406114,3.194329891923853932595347583210722,"{'block': '11861237', 'id': '0x5c67dbb323db039...",{'hourlyRate': '1'},1613394096
...,...,...,...,...,...,...,...,...
846,12094211,42619776.070569356751824501,42875118.83251601892205681663084113,0.00175509626408125720581657952035192,2.955873091756318173928107873662533,"{'block': '12093948', 'id': '0x9942178e932663e...",{'hourlyRate': '1.000040118754967389780035332'},1616490486
847,12094530,42813047.631845043179160483,43068648.37406041594072836345220295,0.001753555143766846954990564281437191,2.962353615013804008291058524106369,"{'block': '12094521', 'id': '0xe7e0c723a9c42ad...",{'hourlyRate': '1.000040118754967389780035332'},1616494741
848,12094860,42858047.631845043179160483,43113660.64550462078443636948548955,0.001753362573510172589058213841461564,2.964695763476635308169088663654856,"{'block': '12094785', 'id': '0xcb0e6161792d9f9...",{'hourlyRate': '1.000040118754967389780035332'},1616498886
849,12095179,42856706.35994837848042724,43112322.12307449993560700248548955,0.001744995039732342403518047564180582,2.973921115260634951080291380999054,"{'block': '12095052', 'id': '0xb70dd24dc998eb3...",{'hourlyRate': '1.000040108709191595229449019'},1616503074


In [41]:
hourlyStats.redemptionPrice.values[-1]

{'block': '12095319',
 'id': '0x9aebee982c3a3c40045a3e09a796dd44393794f83b24d2a0afe990e7f1c14d07-163',
 'value': '3.021866783931796905697155521'}

In [42]:
hourlyStats.redemptionRate.values[-1]

{'hourlyRate': '1.000040108709191595229449019'}

In [43]:
hourlyStats['redemptionPriceActual'] = hourlyStats.redemptionPrice.apply(lambda x: x['value'])

In [45]:
hourlyStats['redemptionRateActual'] = hourlyStats.redemptionRate.apply(lambda x: x['hourlyRate'])

In [46]:
hourlyStats['erc20CoinTotalSupply'] = hourlyStats['erc20CoinTotalSupply'].astype(float)
hourlyStats['globalDebt'] = hourlyStats['globalDebt'].astype(float)
hourlyStats['blockNumber'] = hourlyStats['blockNumber'].astype(int)
hourlyStats['marketPriceEth'] = hourlyStats['marketPriceEth'].astype(float)
hourlyStats['marketPriceUsd'] = hourlyStats['marketPriceUsd'].astype(float)
hourlyStats['redemptionPriceActual'] = hourlyStats['redemptionPriceActual'].astype(float)
hourlyStats['redemptionRateActual'] = hourlyStats['redemptionRateActual'].astype(float)

In [47]:
hourlyStats.describe()

,blockNumber,erc20CoinTotalSupply,globalDebt,marketPriceEth,marketPriceUsd,redemptionPriceActual,redemptionRateActual
count,8.510000e+02,8.510000e+02,8.510000e+02,851.000000,851.000000,851.000000,851.000000
mean,1.197706e+07,3.325319e+07,3.370093e+07,0.001882,3.218255,3.065455,0.999955
std,6.749445e+04,1.420924e+07,1.433379e+07,0.000300,0.444691,0.049259,0.000059
min,1.185731e+07,1.988440e+03,1.988577e+03,0.001600,2.850455,3.013686,0.999796
25%,1.191878e+07,2.749865e+07,2.755863e+07,0.001671,3.009266,3.017865,0.999913
50%,1.197660e+07,3.463165e+07,3.549437e+07,0.001781,3.142389,3.047204,0.999961
75%,1.203514e+07,4.513489e+07,4.532411e+07,0.002053,3.265479,3.117719,1.000004
max,1.209544e+07,4.899876e+07,4.918825e+07,0.004474,8.055382,3.140000,1.000044


## Daily stats - WIP

In [48]:
query_header = '''
query {{
    dailyStats(first: 1000, skip:{}) {{'''

query_tail = '''    
}
}'''

query_body = '''
    timestamp
    globalDebt
    redemptionRate {
      twentyFourHourlyRate
    }
'''



n = 0
results = []
while True:
    print(f'request {n+1}')
    query = query_header.format(n*1000) + query_body + query_tail
    r = requests.post(url, json = {'query':query})
    s = json.loads(r.content)['data']['dailyStats']
    print(f'results {len(s)}')
    results.extend(s)
    print(f'total {len(results)}')
    n += 1
    if len(s) < 1000:
        break
        
dailyStats = pd.DataFrame(results)

dailyStats.head()

request 1
results 38
total 38


,globalDebt,redemptionRate,timestamp
0,1988.508966230755336947218345592486,{'twentyFourHourlyRate': '1'},1613338681
1,2788.522357955470745187436691184972,{'twentyFourHourlyRate': '1'},1613383785
2,7204.841129160410399042582689488635,{'twentyFourHourlyRate': '1'},1613436497
3,10603.09390239356752028390089696083,{'twentyFourHourlyRate': '0.999999085220771157...,1613520522
4,4755099.686882616278066513280682631,{'twentyFourHourlyRate': '0.999999911975837683...,1613606864


## System State - WIP

In [49]:
blocknumbers = hourlyStats.blockNumber.values.tolist()
state = []
for i in blocknumbers:
    query = '''
    {
      systemState(block: {number:%s},id:"current") { 
        coinUniswapPair {
          reserve0
          reserve1
        }
        currentCoinMedianizerUpdate{
          value
        }
        currentRedemptionRate {
          eightHourlyRate
          annualizedRate
          hourlyRate
          createdAt
        }
        currentRedemptionPrice {
          value
        }
        erc20CoinTotalSupply
        globalDebt
        globalDebtCeiling
        totalActiveSafeCount
        coinAddress
        wethAddress
        systemSurplus
        debtAvailableToSettle
      }
    }
    ''' % i
    r = requests.post(url, json = {'query':query})
    s = json.loads(r.content)['data']['systemState']
    state.append(s)
        
systemState = pd.DataFrame(state)

systemState.head()
    

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

## Internal Coin Balances - WIP

balance = []
for i in blocknumbers:
    query = '''
    {
      internalCoinBalances((block: {number:%s},where: {accountHandler: "${GEB_STABILITY_FEE_TREASURY}"}) {
        balance
      }
    }
    ''' % i
    r = requests.post(url, json = {'query':query})
    s = json.loads(r.content)['data']['internalCoinBalances']
    balance.append(s)
        
coinBalances = pd.DataFrame(balance)

coinBalances.head()


## Accounting Engine - WIP

In [52]:
blocknumbers = hourlyStats.blockNumber.values.tolist()
accounts = []
for i in blocknumbers:
    query = '''
    {
      accountingEngine(block: {number:%s},id: "current") { 
        activeDebtAuctions
        activeSurplusAuctions
      }
    }
    ''' % i
    r = requests.post(url, json = {'query':query})
    s = json.loads(r.content)['data']['accountingEngine']
    accounts.append(s)
        
accountingEngine = pd.DataFrame(accounts)

accountingEngine.head()
        

,activeDebtAuctions,activeSurplusAuctions
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0


## Eth Price

In [58]:
blocknumbers = hourlyStats.blockNumber.values.tolist()
eth_price = []
for i in blocknumbers:
    query = '''
    {
      collateralType(block: {number:%s}, id: "ETH-A") {
        currentFsmUpdate {
          value
        }
        currentMedianizerUpdate {
          value
        }
        debtCeiling
        activeLiquidations
        liquidationsStarted
        totalCollateral
        totalAnnualizedStabilityFee
      }
    }
    ''' % i
    r = requests.post(url, json = {'query':query})
    s = json.loads(r.content)['data']['collateralType']
    eth_price.append(s)
        
collateralType = pd.DataFrame(eth_price)

collateralType['blockNumber'] = blocknumbers
collateralType.head()
    

,activeLiquidations,currentFsmUpdate,currentMedianizerUpdate,debtCeiling,liquidationsStarted,totalAnnualizedStabilityFee,totalCollateral,blockNumber
0,0,{'value': '1803.65643044'},{'value': '1803.65643044'},10000000,0,1.019999999999999999972831879,14.892360473369843164,11857308
1,0,{'value': '1803.65643044'},{'value': '1763.97493608'},10000000,0,1.019999999999999999972831879,14.892360473369843164,11860793
2,0,{'value': '1763.97493608'},{'value': '1775.97792296'},10000000,0,1.019999999999999999972831879,14.892360473369843164,11861040
3,0,{'value': '1763.97493608'},{'value': '1780.98836373'},10000000,0,1.019999999999999999972831879,14.892360473369843164,11861337
4,0,{'value': '1763.97493608'},{'value': '1805.79273513'},10000000,0,1.019999999999999999972831879,16.892360473369843164,11861474


In [59]:

collateralType['Eth_price'] = collateralType.currentFsmUpdate.apply(lambda x: x['value'])
collateralType['Eth_price'] = collateralType['Eth_price'].astype(float)


In [64]:
hourlyStats = hourlyStats.merge(collateralType,how='inner',on='blockNumber')

In [65]:
hourlyStats.to_csv('saved_results/RaiLiveGraphData.csv')

## Safes

## Conclusion

Using The Graph, a lot of data about the Rai system can be obtained for analyzing the health of the system. With some data manipulation, these data streams could be intergrated into the Rai cadCAD model to turn it into a true decision support system.